In [1]:
! pip install torchtext==0.6.0

     |████████████████████████████████| 64 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 9.0 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [2]:
%%capture
! python -m spacy download en
! python -m spacy download de

In [3]:
import spacy

spacy_en = spacy.load('en') # english tokenization
spacy_de = spacy.load('de')

In [4]:
# test

tokenized = spacy_en.tokenizer("Hello, today is sunday")

for i, token in enumerate(tokenized):
  print(f"index {i} {token.text}")

index 0 Hello
index 1 ,
index 2 today
index 3 is
index 4 sunday


In [6]:
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

In [5]:
from torchtext.data import Field, BucketIterator

In [7]:
SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [8]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:04<00:00, 301kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 92.7kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 87.8kB/s]


In [9]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [12]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [13]:
SRC.build_vocab(train_dataset, min_freq = 2)
TRG.build_vocab(train_dataset, min_freq = 2)

In [14]:
print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7855
len(TRG): 5893


In [15]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


In [16]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size = BATCH_SIZE,
    device = device
)

In [19]:
for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  print(f"첫 번째 배치 크기: {src.shape}")

  # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
  for i in range(src.shape[1]):
      print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

  # 첫 번째 배치만 확인
  break

첫 번째 배치 크기: torch.Size([128, 29])
인덱스 0: 2
인덱스 1: 45
인덱스 2: 55
인덱스 3: 91
인덱스 4: 2598
인덱스 5: 100
인덱스 6: 6
인덱스 7: 249
인덱스 8: 200
인덱스 9: 28
인덱스 10: 172
인덱스 11: 211
인덱스 12: 652
인덱스 13: 4
인덱스 14: 3
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1


### Multi Head Attention 아키텍처
* 어텐션(attention)은 세 가지 요소를 입력으로 받습니다.
  * 쿼리(queries)
  * 키(keys)
  * 값(values)
  * 현재 구현에서는 Query, Key, Value의 차원이 모두 같습니다.

* 하이퍼 파라미터(hyperparameter)
  * hidden_dim: 하나의 단어에 대한 임베딩 차원
  * n_heads: 헤드(head)의 개수 = scaled  dot-product attention의 개수
  * dropout_ratio: 드롭아웃(dropout) 비율

In [21]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim
    self.n_heads = n_heads
    self.head_dim = hidden_dim // n_heads

    self.fc_q = nn.Linear(hidden_dim, hidden_dim)
    self.fc_k = nn.Linear(hidden_dim, hidden_dim)
    self.fc_v = nn.Linear(hidden_dim, hidden_dim)

    self.fc_o = nn.Linear(hidden_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):
    batch_size = query.shape[0]

    # query : [batch_size, query_len, hidden_dim]
    # key : [bacth_size, query_len, hidden_dim]
    # value : [batch_size, query_len, hidden_dim]

    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    # hidden_dim -> n_heads * head_dim
    # n_heads(h) 개의 서로 다른 어텐션 컨셉을 학습할 수 있도록 유도한다

    # Q: [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]
    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

    # Attention Energy 계산과정
    # energy : [batch_size, n_heads, query_len, key_len]
    energy = torch.matmul(Q, K.permutate(0, 1, 3, 2)) / self.scale

    # 마스크
    if mask is not None:
      energy = energy.masked_fill(mask == 0, -1e10)

    # attention score calculate
    # attention = [batch_size, n_heads, query_len, key_len]
    attention = torch.softmax(energy, dim = -1) # 마지막차원 벡터를 기준으로 소프트맥스


    # x : [batch_size, n_heads, query_len, head_dim]
    x = torch.matmul(self.dropout(attention), V) # key_len == query_len

    # x : [batch_size, query_len, n_heads, head_dim]
    x = x.permute(0, 2, 1, 3).contiguous()

    # x : [batch_size, query_len, hidden_dim]
    x = x.view(batch_size, -1, self.hidden_dim)

    # x : [batch_size, query_len, hidden_dim]
    x = self.fc_o(x)

    return x, attention


### Position-wise Feedforward 아키텍처

* 입력과 출력의 차원이 동일합니다.
* 하이퍼 파라미터(hyperparameter)
  * hidden_dim: 하나의 단어에 대한 임베딩 차원
  * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
  * dropout_ratio: 드롭아웃(dropout) 비율

In [22]:
class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, hidden_dim, pf_dim, dropout_ratio):
    super().__init__()

    self.fc_1 = nn.Linear(hidden_dim, pf_dim)
    self.fc_2 = nn.Linear(pf_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, x):
    # x: [batch_size, seq_len, hidden_dim] 로 입력이 들어온다.

    # x :[batch_size, seq_len, pf_dim]
    x = self.fc_1(x)

    # dropout && relu
    x = self.dropout(torch.relu(x))

    # x : [batch_size, seq_len, hidden_dim]
    x = self.fc_2(x)

    return x


### 인코더(Encoder) 레이어 아키텍처

* 하나의 인코더 레이어에 대해 정의합니다.
    * 입력과 출력의 차원이 같습니다.
    * 이러한 특징을 이용해 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해 사용합니다.

* 하이퍼 파라미터(hyperparameter)
  * hidden_dim: 하나의 단어에 대한 임베딩 차원
  * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
  * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
  * dropout_ratio: 드롭아웃(dropout) 비율
* <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.

In [24]:
class EncoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feadforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)


  def forward(self, src, src_mask):
    # src : [batch_size, src_len, hidden_dim] 으로 입력
    # src_mask : [batch_size, src_len] 으로 입력

    # self attention
    # 필요한 경우, 마스크 행렬을 이용해서 어텐션할 단어를 조절
    _src, _ = self.sefl_attention(src, src, src, src_mask) # forward를 진행한다

    # dropout, residual connection and layer Norm
    # src : [batch_size, src_len, hidden_dim]
    src = self.sefl_attn_layer_norm(src + self.dropout(_src))

    # position-wise feedforward
    _src = self.positionwise_feedforward(src)

    # dropout, residual and layernorm
    # src: [batch_size, src_len, hidden_dim]
    src = self.ff_layer_norm(src + self.dropout(_src))

    return src



### 인코더(Encoder) 아키텍처

* 전체 인코더 아키텍처를 정의합니다.
* 하이퍼 파라미터(hyperparameter)
  * input_dim: 하나의 단어에 대한 원 핫 인코딩 차원
  * hidden_dim: 하나의 단어에 대한 임베딩 차원
  * n_layers: 내부적으로 사용할 인코더 레이어의 개수
  * n_heads: 헤드(head)의 개수 = scaled * * * dot-product attention의 개수
  * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
  * dropout_ratio: 드롭아웃(dropout) 비율
  * max_length: 문장 내 최대 단어 개수
* 원본 논문과는 다르게 위치 임베딩(positional embedding)을 학습하는 형태로 구현합니다.
  * BERT와 같은 모던 트랜스포머 아키텍처에서 사용되는 방식입니다.
* <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.

In [25]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):

      # src: [batch_size, src_len]
      # src_mask: [batch_size, src_len]

      batch_size = src.shape[0]
      src_len = src.shape[1]

      # pos: [batch_size, src_len]
      pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

      # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
      # src: [batch_size, src_len, hidden_dim]
      src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))


      # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
      for layer in self.layers:
          src = layer(src, src_mask)

      # src: [batch_size, src_len, hidden_dim]

      return src # 마지막 레이어의 출력을 반환

### 디코더(Decoder) 레이어 아키텍처

* 하나의 디코더 레이어에 대해 정의합니다.
  * 입력과 출력의 차원이 같습니다.
  * 이러한 특징을 이용해 트랜스포머의 디코더는 디코더 레이어를 여러 번 중첩해 사용합니다.
  * 디코더 레이어에서는 두 개의 Multi-Head Attention 레이어가 사용됩니다.
* 하이퍼 파라미터(hyperparameter)
  * hidden_dim: 하나의 단어에 대한 임베딩 차원
  * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
  * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
  * dropout_ratio: 드롭아웃(dropout) 비율
* 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.
* 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용합니다.

In [26]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # trg: [batch_size, trg_len, hidden_dim]
        # enc_src: [batch_size, src_len, hidden_dim]
        # trg_mask: [batch_size, trg_len]
        # src_mask: [batch_size, src_len]

        # self attention
        # 자기 자신에 대하여 어텐션(attention)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # dropout, residual connection and layer norm
        # trg: [batch_size, trg_len, hidden_dim]
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        

        # encoder attention
        # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
        # attention: [batch_size, n_heads, trg_len, src_len]
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection and layer norm
        # trg: [batch_size, trg_len, hidden_dim]
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual and layer norm
        # trg: [batch_size, trg_len, hidden_dim]
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        
    
        return trg, attention

### 디코더(Decoder) 아키텍처

* 전체 디코더 아키텍처를 정의합니다.
* 하이퍼 파라미터(hyperparameter)
  * output_dim: 하나의 단어에 대한 원 핫 인코딩 차원
  * hidden_dim: 하나의 단어에 대한 임베딩 차원
  * n_layers: 내부적으로 사용할 인코더 레이어의 개수
  * n_heads: 헤드(head)의 개수 = scaled dot-product attention의 개수
  * pf_dim: Feedforward 레이어에서의 내부 임베딩 차원
  * dropout_ratio: 드롭아웃(dropout) 비율
  * max_length: 문장 내 최대 단어 개수
* 원본 논문과는 다르게 위치 임베딩(positional embedding)을 학습하는 형태로 구현합니다.
  * BERT와 같은 모던 트랜스포머 아키텍처에서 사용되는 방식입니다.
* Seq2Seq과는 마찬가지로 실제로 추론(inference) 시기에서는 디코더를 반복적으로 넣을 필요가 있습니다.
  * 학습(training) 시기에서는 한 번에 출력 문장을 구해 학습할 수 있습니다.
* 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정합니다.
* 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용합니다.

In [27]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

      # trg: [batch_size, trg_len]
      # enc_src: [batch_size, src_len, hidden_dim]
      # trg_mask: [batch_size, trg_len]
      # src_mask: [batch_size, src_len]

      batch_size = trg.shape[0]
      trg_len = trg.shape[1]

      # pos: [batch_size, trg_len]
      pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

      
      # trg: [batch_size, trg_len, hidden_dim]
      trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

      for layer in self.layers:
        # 소스 마스크와 타겟 마스크 모두 사용
        # trg: [batch_size, trg_len, hidden_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]
        trg, attention = layer(trg, enc_src, trg_mask, src_mask)

      

      output = self.fc_out(trg)

      # output: [batch_size, trg_len, output_dim]

      return output, attention

### 트랜스포머(Transformer) 아키텍처
* 최종적인 전체 트랜스포머(Transformer) 모델을 정의합니다.
* 입력이 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장을 생성합니다.


In [28]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
    def make_src_mask(self, src):

        # src: [batch_size, src_len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask: [batch_size, 1, 1, src_len]

        return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
    def make_trg_mask(self, trg):

        # trg: [batch_size, trg_len]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 0 0
        1 1 1 0 0
        """
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

        # trg_pad_mask: [batch_size, 1, 1, trg_len]

        trg_len = trg.shape[1]

        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

        # trg_sub_mask: [trg_len, trg_len]

        trg_mask = trg_pad_mask & trg_sub_mask

        # trg_mask: [batch_size, 1, trg_len, trg_len]

        return trg_mask

    def forward(self, src, trg):

        # src: [batch_size, src_len]
        # trg: [batch_size, trg_len]

        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        # src_mask: [batch_size, 1, 1, src_len]
        # trg_mask: [batch_size, 1, trg_len, trg_len]

        enc_src = self.encoder(src, src_mask)

        # enc_src: [batch_size, src_len, hidden_dim]

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output: [batch_size, trg_len, output_dim]
        # attention: [batch_size, n_heads, trg_len, src_len]

        return output, attention

### 학습(Training)
* 하이퍼 파라미터 설정 및 모델 초기화

In [29]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [30]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)


In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,038,853 trainable parameters


In [32]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7855, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feadforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
 

In [33]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [34]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [35]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [36]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [37]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')


NotImplementedError: ignored